## Ejemplo Backtesting.py

In [48]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.lib import resample_apply


from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, cash=10000,
              commission=.002, exclusive_orders=True)

output = bt.run()
bt.plot()

In [49]:
output

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     94.2737
Equity Final [$]                      68935.1
Equity Peak [$]                       68991.2
Return [%]                            589.351
Buy & Hold Return [%]                 703.458
Max. Drawdown [%]                    -33.0822
Avg. Drawdown [%]                    -5.58151
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   93
Win Rate [%]                          53.7634
Best Trade [%]                        57.1193
Worst Trade [%]                      -16.6299
Avg. Trade [%]                        1.95909
Max. Trade Duration         121 days 00:00:00
Avg. Trade Duration          32 days 00:00:00
Profit Factor                         2.12576
Expectancy [%]                        6.91487
SQN                               

## Ejemplo MACD con datos IBM

In [1]:
import pandas as pd

header_list = ["Date", "Time", "Open", "High", "Low", "Close", "Volume"]
IBM = pd.read_csv('../Resources/IBM_adjusted.csv', names=header_list)
IBM['Datetime'] = pd.to_datetime(IBM['Date'] + ' ' + IBM['Time'])
IBM = IBM.set_index('Datetime')
IBM = IBM[['Open', 'High', 'Low', 'Close', 'Volume']]
IBM1 = IBM[-5000:]
IBM2 = IBM[-20000:-15000]

In [43]:
import pandas as pd

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

def MACD(array, n1, n2):
    fastEMA = pd.Series(array).ewm(span=n1, adjust=False).mean()
    slowEMA = pd.Series(array).ewm(span=n2, adjust=False).mean()
    return fastEMA - slowEMA
    
def SIGNAL(array, n):
    return pd.Series(array).ewm(span=n, adjust=False).mean()

def EMA(array, n):
    return pd.Series(array).ewm(span=n, adjust=False).mean()


In [46]:
class MACDStrategy(Strategy):
    status = 0
    stoploss = 0
    closeprofit = 0
    slrate = 25
    cprate = 50
    buyInfo = ['', 0]
    
    def init(self):
        # Compute moving averages the strategy demands
        self.macd = self.I(MACD, self.data.Close, 12, 26)
        self.signal = self.I(SIGNAL, self.macd, 9)
        self.ema200 = self.I(EMA, self.data.Close, 200)
        self.slopeEMA200 = np.diff(self.ema200)
        
        
            
    def next(self):
        
        if crossover(self.macd, self.signal) and self.macd > 0:
            if self.slopeEMA200[-1] > self.slopeEMA200[-2]:
                self.buy()
                self.stoploss = self.data.Close[-1]*(1-self.slrate/10000)
                self.closeprofit = self.data.Close[-1]*(1+self.cprate/10000)
                self.buyInfo = [self.data.index[-1], self.data.Close[-1]]
                #print(f'Current Price: {self.data.Close[-1]}, Stoploss: {self.stoploss}, Closeprofit: {self.closeprofit}')

        elif (self.data.Close[-1] >= self.closeprofit or self.data.Close[-1] <= self.stoploss):
            self.position.close()
            
            

        

In [47]:
bt = Backtest(IBM1, MACDStrategy, cash=10000,
              commission=.002, exclusive_orders=True)

output = bt.run()
bt.plot()

In [38]:
output

Start                     2020-07-29 11:12:00
End                       2020-08-13 19:49:00
Duration                     15 days 08:37:00
Exposure Time [%]                       21.64
Equity Final [$]                      9680.42
Equity Peak [$]                         10063
Return [%]                           -3.19582
Buy & Hold Return [%]                 1.67303
Max. Drawdown [%]                    -3.80217
Avg. Drawdown [%]                   -0.310221
Max. Drawdown Duration       13 days 04:45:00
Avg. Drawdown Duration        0 days 18:50:00
# Trades                                   19
Win Rate [%]                          36.8421
Best Trade [%]                       0.395738
Worst Trade [%]                     -0.714119
Avg. Trade [%]                      -0.171809
Max. Trade Duration           2 days 18:07:00
Avg. Trade Duration           0 days 07:47:00
Profit Factor                        0.412651
Expectancy [%]                       0.411365
SQN                               

In [39]:
optimized = bt.optimize(slrate=range(15, 75, 10),
                  cprate=range(15, 75, 10))

In [40]:
optimized

Start                     2020-07-29 11:12:00
End                       2020-08-13 19:49:00
Duration                     15 days 08:37:00
Exposure Time [%]                       33.14
Equity Final [$]                      10125.3
Equity Peak [$]                       10275.6
Return [%]                             1.2535
Buy & Hold Return [%]                 1.67303
Max. Drawdown [%]                    -1.74983
Avg. Drawdown [%]                   -0.226164
Max. Drawdown Duration        6 days 00:04:00
Avg. Drawdown Duration        0 days 07:52:00
# Trades                                   13
Win Rate [%]                          61.5385
Best Trade [%]                        1.04447
Worst Trade [%]                     -0.734274
Avg. Trade [%]                       0.110976
Max. Trade Duration           2 days 20:33:00
Avg. Trade Duration           0 days 14:38:00
Profit Factor                         1.43046
Expectancy [%]                       0.638513
SQN                               

In [41]:
bt.plot()
